<a href="https://colab.research.google.com/github/r1marcus/Embedded_KI_Workshop/blob/main/Pruning_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing Pruning

In [1]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 5.7MB/s 


In [2]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tempfile
#import tensorflow_model_optimization as tfmot
import numpy as np


# Model configuration
img_width, img_height = 28, 28
batch_size = 250
no_epochs = 10
no_classes = 10
validation_split = 0.2
verbosity = 1

# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()
input_shape = (img_width, img_height, 1)

# Reshape data for ConvNet
input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize [0, 255] into [0, 1]
input_train = input_train / 255
input_test = input_test / 255

# Convert target vectors to categorical targets
target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(input_train, target_train,
          batch_size=batch_size,
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/10
192/192 [==============================] - 40s 204ms/step - loss: 0.6938 - accuracy: 0.7977 - val_loss: 0.0979 - val_accuracy: 0.9718
Epoch 2/10
192/192 [==============================] - 39s 202ms/step - loss: 0.1021 - accuracy: 0.9678 - val_loss: 0.0611 - val_accuracy: 0.9834
Epoch 3/10
192/192 [==============================] - 39s 203ms/step - loss: 0.0720 - accuracy: 0.9771 - val_loss: 0.0485 - val_accuracy: 0.9863
Epoch 4/10
192/192 [==============================] - 39s 203ms/step - loss: 0.0577 - accuracy: 0.9813 - val_loss: 0.0431 - val_accuracy: 0.9879
Epoch 5/10
192/192 [==============================] - 39s 202ms/step - loss: 0.0432 - accuracy: 0.9868 - val_loss: 0.0421 - val_accuracy: 0.9885
Epoch 6/10
192/192 [==============================] - 39s 201ms/step - loss: 0.0372 - accuracy: 0.9882 - val_loss: 0.0388 - val_accuracy: 0.9896
Epoch 7/10
192/192 [==============================] - 39s 202ms/s

# Modell Speichern
Stellen Sie außerdem sicher, dass Sie Ihr Modell in einer temporären Datei speichern, so dass Sie später die Größen des ursprünglichen und des geprunten Modells vergleichen können:

In [3]:

from tensorflow import keras
import tensorflow_model_optimization as tfmot
_, keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model, keras_file, include_optimizer=False)
print(f'Baseline model saved: {keras_file}')

Baseline model saved: /tmp/tmp2jfznccm.h5


# Loading and configuring pruning

Es lädt die prune_low_magnitude Funktionalität von TensorFlow (Tfmot.sparsity.keras.prune_low_magnitude, n.d.). prune_low_magnitude modifiziert einfach einen Layer, indem es ihn für das Pruning bereit macht. Dies geschieht, indem ein Keras-Modell mit Pruning-Funktionalität umhüllt wird, genauer gesagt, indem sichergestellt wird, dass die Schichten des Modells beschneidbar sind. Damit wird nur die Funktionalität geladen, wir werden sie später tatsächlich aufrufen.

Nach dem Laden des Pruning-Wrappers werden wir die Pruning-Konfiguration festlegen:

In [4]:
# Finish pruning after 5 epochs
pruning_epochs = 5
num_images = input_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * pruning_epochs

# Define pruning configuration
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.40,
                                                               final_sparsity=0.70,
                                                               begin_step=0,
                                                               end_step=end_step)
}
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Hier geschieht Folgendes:

Wir konfigurieren die Länge des Pruning-Prozesses über die Anzahl der Epochen, für die das Modell prunen soll, und nehmen eine Feinabstimmung vor.
Wir laden die Anzahl der Bilder, die in unserem Trainingssatz verwendet werden, abzüglich der Validierungsdaten.
Wir berechnen den end_step unseres Pruning-Prozesses anhand der Stapelgröße, der Anzahl der Bilder sowie der Anzahl der Epochen.
Anschließend definieren wir die Konfiguration für den Pruning-Vorgang über pruning_params. Wir definieren einen Pruning-Zeitplan unter Verwendung von PolynomialDecay, was bedeutet, dass die Sparsamkeit des Modells mit zunehmender Anzahl von Epochen zunimmt. Zu Beginn wird das Modell auf 40 % Sparsamkeit eingestellt und wird zunehmend spärlicher, bis es schließlich 70 % erreicht. Wir beginnen bei 0 und enden bei end_step.
Schließlich rufen wir die Funktion "prune_low_magnitude" (die das beschneidbare Modell erzeugt) mit unserem Ausgangsmodell und den definierten "pruning_params" auf.



# Starten des Beschneidungsvorgangs
Nachdem wir den Pruning-Prozess konfiguriert haben, können wir das Modell neu kompilieren (dies ist notwendig, da wir die Pruning-Funktionalität hinzugefügt haben) und den Pruning-Prozess starten. Wir müssen hier den UpdatePruningStep-Callback verwenden, da er die Optimierungsaktivitäten an den Pruning-Prozess weitergibt (Tfmot.sparsity.keras.UpdatePruningStep, o.J.).

In [5]:
# Recompile the model
model_for_pruning.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Model callbacks
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

# Fitting data
model_for_pruning.fit(input_train, target_train,
                      batch_size=batch_size,
                      epochs=pruning_epochs,
                      verbose=verbosity,
                      callbacks=callbacks,
                      validation_split=validation_split)

Epoch 1/5
192/192 [==============================] - 42s 205ms/step - loss: 0.0214 - accuracy: 0.9931 - val_loss: 0.0292 - val_accuracy: 0.9911
Epoch 2/5
192/192 [==============================] - 40s 207ms/step - loss: 0.0247 - accuracy: 0.9918 - val_loss: 0.0295 - val_accuracy: 0.9913
Epoch 3/5
192/192 [==============================] - 40s 206ms/step - loss: 0.0209 - accuracy: 0.9931 - val_loss: 0.0282 - val_accuracy: 0.9923
Epoch 4/5
192/192 [==============================] - 40s 206ms/step - loss: 0.0185 - accuracy: 0.9946 - val_loss: 0.0299 - val_accuracy: 0.9916
Epoch 5/5
192/192 [==============================] - 39s 205ms/step - loss: 0.0158 - accuracy: 0.9954 - val_loss: 0.0275 - val_accuracy: 0.9920


# Messen der Effektivität des Beschneidens
Sobald das Pruning abgeschlossen ist, müssen wir seine Effektivität messen. Das können wir auf zwei Arten tun:

Indem man misst, wie sehr sich die Leistung im Vergleich zu vor dem Pruning verändert hat;
Indem wir messen, wie sehr sich die Modellgröße im Vergleich zur Zeit vor dem Pruning verändert hat.
Wir tun dies mit den folgenden Codezeilen:

In [6]:
# Generate generalization metrics
score_pruned = model_for_pruning.evaluate(input_test, target_test, verbose=0)
print(f'Pruned CNN - Test loss: {score_pruned[0]} / Test accuracy: {score_pruned[1]}')
print(f'Regular CNN - Test loss: {score[0]} / Test accuracy: {score[1]}')

Pruned CNN - Test loss: 0.021241622045636177 / Test accuracy: 0.9926000237464905
Regular CNN - Test loss: 0.023596348240971565 / Test accuracy: 0.9914000034332275


Diese sind einfach. Sie werten das beschnittene Modell mit den Testdaten aus und geben anschließend das Ergebnis aus, ebenso wie das (zuvor erhaltene) Ergebnis des ursprünglichen Modells.

Als Nächstes exportieren wir es erneut - genau wie zuvor - um sicherzustellen, dass wir es vergleichen können:

In [7]:
# Export the model
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print(f'Pruned model saved: {keras_file}')

Pruned model saved: /tmp/tmp2jfznccm.h5


Anschließend können wir (dank Pruning Keras Example (n.d.)) die Größe des Keras-Modells vergleichen. Um die Vorteile von Pruning zu verdeutlichen, müssen wir einen Komprimierungsalgorithmus wie gzip verwenden, wonach wir die Größen beider Modelle vergleichen können. Erinnern Sie sich daran, dass Pruning Sparsity erzeugt und dass dünn besetzte Matrizen bei der Komprimierung sehr effizient gespeichert werden können. Deshalb ist gzips für Demonstrationszwecke nützlich. Wir erstellen zunächst eine def, die für die Komprimierung verwendet werden kann, und rufen sie anschließend zweimal auf:

In [8]:
# Measuring the size of your pruned model
# (source: https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras#fine-tune_pre-trained_model_with_pruning)

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))

Size of gzipped baseline Keras model: 1601249.00 bytes
Size of gzipped pruned Keras model: 679469.00 bytes


# Laufzeit-Ergebnis
Nun ist es an der Zeit, es auszuführen. Speichern Sie Ihre Datei als z.B. pruning.py, und führen Sie sie in einer Python-Umgebung aus, in der Sie sowohl tensorflow 2.x als auch numpy und das tensorflow_model_optimization toolkit installiert haben.

Zuerst wird das reguläre Training gestartet, gefolgt vom Pruning-Prozess, und dann wird die Effektivität auf dem Bildschirm angezeigt. Zunächst in Bezug auf die Modellleistung (d. h. Verlust und Genauigkeit):

Pruned CNN - Testverlust: 0.0218335362634185 / Testgenauigkeit: 0.9923999905586243
Regulärer CNN - Testverlust: 0.02442687187876436 / Testgenauigkeit: 0.9915000200271606
Das beschnittene Modell schneidet sogar etwas besser ab als das reguläre Modell. Das liegt wahrscheinlich daran, dass wir das ursprüngliche Modell nur für 10 Epochen trainiert haben und danach mit dem Pruning fortgefahren sind. Es ist sehr gut möglich, dass das Modell noch nicht konvergiert war, sondern sich im Pruning-Prozess weiter in Richtung Konvergenz bewegt hat. Oftmals verschlechtert sich die Performance ein wenig, aber das sollte nur geringfügig sein.

Dann, in Bezug auf die Modellgröße:

Größe des gezippten Keras-Basismodells: 1601609.00 Bytes
Größe des gzipped pruned Keras-Modells: 679958.00 bytes
Code-Sprache: CSS (css)
Das Pruning hat unser Modell definitiv kleiner gemacht - 2,35 Mal!

# Was ist Quantisierung?
Quantisierung, kurz gesagt, bedeutet, die Zahlendarstellung Ihres Machine-Learning-Modells zu ändern (ob das Gewichte oder auch Aktivierungen sind), um sie kleiner zu machen.

Standardmäßig arbeiten TensorFlow und Keras mit dem float32-Format. Mit 32-Bit Fließkommazahlen ist es möglich, wirklich große Zahlen mit großer Präzision zu speichern. Allerdings macht die Tatsache, dass 32 Bit verwendet werden können, das Modell nicht so effizient in Bezug auf die Speicherung - und auch nicht in Bezug auf die Geschwindigkeit (Float-Operationen werden normalerweise am besten auf GPUs ausgeführt, und das ist umständlich, wenn Sie Ihr Modell im Feld einsetzen wollen).

Quantisierung bedeutet, diese Zahlendarstellung zu ändern. Mit der Float16-Quantisierung kann man z. B. Teile des Modells vom Float32- in das Float16-Format konvertieren - das reduziert die Modellgröße um etwa 50 %, ohne viel Leistung zu verlieren. Andere Ansätze erlauben die Quantisierung in das int8-Format (möglicherweise mit einem ziemlichen Performance-Verlust, aber einem 4-fachen Größengewinn) oder in ein kombiniertes int8/int16-Format (das Beste aus beiden Welten). Glücklicherweise ist es auch möglich, Ihr Modell quantisierungssensitiv zu machen, was bedeutet, dass es die Quantisierung während des Trainings simuliert, so dass sich die Schichten bereits an den durch die Quantisierung entstehenden Leistungsverlust anpassen können.

Kurz gesagt, nachdem das Modell beschnitten wurde - d. h. von nicht beitragenden Gewichten befreit wurde - können wir nachträglich Quantisierung hinzufügen. Sie sollte das Modell auf zusammengesetzte Weise noch kleiner machen: Eine 2,35-fache Größenreduktion sollte theoretisch bei Verwendung der int8-Quantisierung eine 4 x 2,35 = 9,4-fache Größenreduktion bedeuten!

# Hinzufügen von Quantisierung zu unserem Keras-Beispiel
Schauen wir uns nun an, wie wir Quantisierung zu einem beschnittenen TensorFlow-Modell hinzufügen können. Genauer gesagt, fügen wir eine dynamische Bereichsquantisierung hinzu, die die Gewichte quantisiert, aber nicht unbedingt die Modellaktivierungen.
Dieser Konverter konvertiert Ihr TensorFlow-Modell in ein TensorFlow-Lite-Äquivalent, gegen das die Quantisierung laufen wird. Das Konvertieren des Modells in ein Lite-Modell erlaubt uns, einen Modell-Optimierer zu spezifizieren - in unserem Fall DEFAULT oder Dynamic Range Quantization. Schließlich konvertieren() wir das Modell:

In [9]:
# Convert into TFLite model and convert with DEFAULT (dynamic range) quantization
stripped_model = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
converter = tensorflow.lite.TFLiteConverter.from_keras_model(stripped_model)
converter.optimizations = [tensorflow.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpobwfs74q/assets


Beachten Sie, dass wir zuerst die Pruning-Wrapper aus dem Modell entfernen müssen, indem wir ein stripped_model erstellen. Wenn die Quantisierung des Modells abgeschlossen ist, können wir es speichern und seine Größe ausdrucken, um zu sehen, wie sehr sich die Situation verbessert hat:

In [11]:
# Save quantized model
_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(tflite_model)
  
# Additional details
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

Size of gzipped pruned and quantized TFlite model: 190696.00 bytes


...Bedeutung:

Größenverbesserung Original -> Pruning: 2,35x

Größenverbesserung Pruning -> Quantisierung: 3.64x

Gesamtgrößenverbesserung Pruning + Quantisierung: 8.58x
Fast 9 mal kleiner! 